In [6]:
import cv2
import numpy as np
from matplotlib import pyplot as plt 
import random ,string

img= cv2.imread('img/5.jpg')
#print(img.shape)

def show_image(val):
    for i,s in enumerate(val):
        cv2.imshow('Hello World'+ str(i),s)    
    cv2.waitKey()
    cv2.destroyAllWindows()

In [2]:
#gray color

gr=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

show_image([gr])


gr.shape


(400, 300)

In [44]:
h, w = img.shape[:2]

print(h,w)

trns= np.float32([[1,0,w/4],[0,1,h/4]])

rt= cv2.getRotationMatrix2D((w/2,h/2),22,1)

#img_t= cv2.transpose(img)

img_t=cv2.warpAffine(img,rt,(w,h))

show_image(img_t)

400 300


In [66]:
#resizing INTER_LANCZOS4, cubic is best one
#https://stackoverflow.com/questions/3112364/how-do-i-choose-an-image-interpolation-method-emgu-opencv
img= cv2.imread('img/1.jpg')
i1=cv2.resize(img,(700,700), interpolation = cv2.INTER_LINEAR  )
i2=cv2.resize(img,(500,500), interpolation = cv2.INTER_LANCZOS4 )
i3=cv2.pyrDown(cv2.pyrDown(cv2.pyrDown(img)))
show_image([i1,i2,i3])

In [81]:
#cropping
img1=cv2.rectangle(img, (55,55),(500,500),255)

cropped= img[200:843, 222:1000]

show_image([cropped,img1])

In [107]:
#blurring
img= cv2.imread('img/5.jpg')
img=cv2.resize(img,(500,500), interpolation = cv2.INTER_LANCZOS4 )

#manual
kernal= np.ones((3,3),np.float32)*1/2
blur= cv2.filter2D(img, -1,kernal)

#auto

b1= cv2.blur(img,(2,2))
b2= cv2.GaussianBlur(img,(3,3),0)
b3= cv2.bilateralFilter(img,9,75,75)
b4= cv2.fastNlMeansDenoisingColored(img,None,6,6,7,21)
show_image([img,b3])

In [112]:
#image sharpening

kernal = np.array([
    [-1,-1,-1],
    [-1,9,-1],
    [-1,-1,-1]
])

sh= cv2.filter2D(img, -1, kernal)

show_image([img, sh])

In [29]:
#threashholding  **

img= cv2.imread('img/1.jpg')
img=cv2.resize(img,(800,800), interpolation = cv2.INTER_LANCZOS4 )

#img= cv2.bilateralFilter(img,9,75,75)
img= cv2.GaussianBlur(img,(3,3),0)

gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#ret,thrd= cv2.threshold(gray_img, 150, 255,cv2.THRESH_TOZERO)
#thrd= cv2.adaptiveThreshold(gray_img, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,3,5)
ret,thrd= cv2.threshold(gray_img, 0, 255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

show_image([gray_img,thrd])
#gray_img

In [35]:
#dialtion and erosion 

kernal = np.ones((5,5),np.uint8)

#erosion
er= cv2.erode(img,kernal,iterations =1)
#dial
di= cv2.dilate(img,kernal,iterations =1)

#morpho

opening = cv2.morphologyEx(img,cv2.MORPH_OPEN,kernal)
closing = cv2.morphologyEx(img,cv2.MORPH_CLOSE,kernal)

show_image([img,opening,closing])

In [58]:
#edge detection
img= cv2.imread('img/nk.jpg')
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.resize(img,(400,500), interpolation = cv2.INTER_LANCZOS4 )


img1= cv2.Canny(img, 100,200)

show_image([img,img1])

#cv2.getPerspectiveTransform

In [29]:
# finding contours
img= cv2.imread('img/4.jpg')
img=cv2.resize(img,(500,400), interpolation = cv2.INTER_LANCZOS4 )
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#blur
gray= cv2.GaussianBlur(gray,(3,3),0)
#ret,gray= cv2.threshold(gray, 50, 255,cv2.THRESH_TOZERO)

#show_image([gray,img])

gray= cv2.Canny(gray, 100,200)
cnts, hr = cv2.findContours(gray.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)


def get_cont_areas(cntrs):
    areas=[]
    for c in cntrs:
        areas.append(cv2.contourArea(c))
    return areas


#actual image cont draw
img1=img.copy()
cv2.drawContours(img1 , cnts,-1,(0,255,0),3)

#blank image cotours draw
blank_img= np.zeros((gray.shape[0],gray.shape[1],3))  
blank_img1= blank_img.copy()
cv2.drawContours(blank_img1 , cnts,-1,(0,255,0),3)


#show max cot.
max_cons = sorted(cnts,key=cv2.contourArea,reverse=True)[:]
img2=img.copy()
cv2.drawContours(img2 , max_cons,-1,(0,255,0),3)
print(get_cont_areas(max_cons[:10]))

show_image([blank_img1,img2])



#finding top contours and convexhull 
def showContoursInSeries():
    for cnt in max_cons[:2]:
        hull= cv2.convexHull(cnt)
        approx=cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt, True),True)
        print(len(approx)) #no of polygons
        img3=img.copy()
        cv2.drawContours(img3 ,[hull],-1,(0,255,0),3)
        show_image([img3])
        
showContoursInSeries()




[188.5, 131.5, 129.5, 121.5, 121.0, 91.0, 87.5, 85.0, 70.0, 67.5]
18
15


In [29]:
#blob detections
import cv2

#skin detection option possible but not good
img= cv2.imread('img/Sunflowers.jpg',cv2.IMREAD_GRAYSCALE)
img=cv2.resize(img,(500,600), interpolation = cv2.INTER_LANCZOS4 )


params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 10
params.maxThreshold = 20000
params.filterByArea = True
detector=cv2.SimpleBlobDetector_create(params)



keypoints=detector.detect(img)

#draw detect blobs
blank = np.zeros((2,2))
blobs=cv2.drawKeypoints(img,keypoints,blank,(0,0,255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


show_image([img,blobs])

In [35]:
#image features detection ##pogo cam
#use best preformer sift and surf
import cv2

img= cv2.imread('img/1.jpg',cv2.IMREAD_GRAYSCALE)
img=cv2.resize(img,(600,600), interpolation = cv2.INTER_LANCZOS4 )

orb= cv2.ORB_create(500,1.5)
keypoints,desc=orb.detectAndCompute(img, None)

img1= cv2.drawKeypoints(img,keypoints,outImage=None,color=(255,0,255))

show_image([img1])